Automated Computational Framework for Multi-Element Urban Ventilation Assessment: Semantic Segmentation and CFD-Ready Geometry Reconstruction
<br> 作者：耿晓天

## 三维纹理网格特征提取计算
输入路径下需同时包含obj, mtl, jpg，支持多纹理图集

In [ ]:
import pandas as pd
import os
import open3d as o3d
import numpy as np
from plyfile import PlyData, PlyElement
from sklearn.neighbors import NearestNeighbors
from meshfea import MeshFeature_Extraction as MFE
from tqdm import tqdm
import h5py

obj_file_path = r"I:\CFD\highbuilding\mesh"
output_dir = r"J:\paper_cfd_code\test"

def process_obj_feature(obj_file_path):
    print(f"Processing: {obj_file_path}")
    try:
        mesh = o3d.io.read_triangle_mesh(obj_file_path)
    except Exception as e:
        print(f"Error reading {obj_file_path}: {e}")
        return

    if mesh.is_empty():
        print(f"Mesh is empty or failed to load: {obj_file_path}")
        return

    try:
        # 构造 MeshFeature_Extraction 对象
        mesh_features = MFE(obj_file_path)
    except Exception as e:
        print(f"Error initializing features for {obj_file_path}: {e}")
        return
    # ---------------特征提取---------------------
    # 提取顶点与三角形索引
    vertices = np.asarray(mesh.vertices)
    triangles = np.asarray(mesh.triangles)
    # 计算每个三角形的中心点（points）及法向量（normal）
    triangle_centroids = np.mean(vertices[triangles], axis=1)  # shape: (num_faces, 3)
    mesh.compute_triangle_normals()
    cen_normal = np.asarray(mesh.triangle_normals)   
    # 初始化各子特征计算对象
    geometric_features = MFE.Geometric_Features(mesh_features)
    elevation_features = MFE.Elevation_Features(mesh_features)
    color_features = MFE.Color_Features(mesh_features)

    # --- 计算面（即三角形）级别的特征 ---

    try:
        areas = geometric_features.mesh_area()
        _area_mean, _area_variance = geometric_features.calculate_triangle_area_variance_and_mean()
        _flatness = geometric_features.mesh_flatness()
        _density = geometric_features.mesh_density()
        _verticality = geometric_features.mesh_verticality()
        _masball_radius = geometric_features.triangleCenter_masb_raduis()
        _masball_radius_mean, _masball_radius_variance = geometric_features.masb_radius_mean_var()
        _height = elevation_features.pointHeight2Ground()
        _mesh_hvi = color_features.calculate_triangle_hvi()
        hsv_hist = color_features.calculate_triangle_hsv_histograms()
        triangle_hsv_stats = color_features.calculate_triangle_hsv_mean_std()
    except Exception as e:
        print(f"Error computing features for {obj_file_path}: {e}")
        return

    num_faces = len(triangles)
    if len(areas) != num_faces:
        print(f"Mismatch: computed {len(areas)} faces != {num_faces} triangles")
        return

    points = np.array([np.array(p) for p in triangle_centroids]) # shape: (num_faces, 3) # 3 0-2
    normal = np.array([np.array(n) for n in cen_normal]) # shape: (num_faces, 3) # 3 3-5
    area = np.expand_dims(areas, axis=1)  # shape: (num_faces, 1) # 1 6
    area_mean = np.expand_dims(_area_mean, axis=1)  # shape: (num_faces, 1) # 1 7
    area_variance = np.expand_dims(_area_variance, axis=1)  # shape: (num_faces, 1) # 1 8
    flatness = np.expand_dims(_flatness, axis=1)  # shape: (num_faces, 1) # 1 9 
    density = np.expand_dims(_density, axis=1)  # shape: (num_faces, 1) # 1 10
    verticality = np.expand_dims(_verticality, axis=1)  # shape: (num_faces, 1) # 1 11
    masball_radius = np.expand_dims(_masball_radius, axis=1)  # shape: (num_faces, 1) # 1 12
    masball_radius_mean = np.expand_dims(_masball_radius_mean, axis=1)  # shape: (num_faces, 1) # 1 13
    masball_radius_variance = np.expand_dims(_masball_radius_variance, axis=1)  # shape: (num_faces, 1) # 1 14
    height = np.expand_dims(_height, axis=1)  # shape: (num_faces, 1) # 1 15
    mesh_hvi = np.expand_dims(_mesh_hvi, axis=1)  # shape: (num_faces, 1) # 1 16 
    
    # Expand all hsv_hist features (assuming they are 1D arrays)
    hsv_hist_0 = np.expand_dims(np.asarray(hsv_hist)[:,0], axis=1)  # shape: (num_faces, 1) # 1 17
    hsv_hist_1 = np.expand_dims(np.asarray(hsv_hist)[:,1], axis=1)  # shape: (num_faces, 1) # 1 18
    hsv_hist_2 = np.expand_dims(np.asarray(hsv_hist)[:,2], axis=1)  # shape: (num_faces, 1) # 1 19
    hsv_hist_3 = np.expand_dims(np.asarray(hsv_hist)[:,3], axis=1)  # shape: (num_faces, 1) # 1 20
    hsv_hist_4 = np.expand_dims(np.asarray(hsv_hist)[:,4], axis=1)  # shape: (num_faces, 1) # 1 21
    hsv_hist_5 = np.expand_dims(np.asarray(hsv_hist)[:,5], axis=1)  # shape: (num_faces, 1) # 1 22
    hsv_hist_6 = np.expand_dims(np.asarray(hsv_hist)[:,6], axis=1)  # shape: (num_faces, 1) # 1 23
    hsv_hist_7 = np.expand_dims(np.asarray(hsv_hist)[:,7], axis=1)  # shape: (num_faces, 1) # 1 24
    hsv_hist_8 = np.expand_dims(np.asarray(hsv_hist)[:,8], axis=1)  # shape: (num_faces, 1) # 1 25
    hsv_hist_9 = np.expand_dims(np.asarray(hsv_hist)[:,9], axis=1)  # shape: (num_faces, 1) # 1 26
    hsv_hist_10 = np.expand_dims(np.asarray(hsv_hist)[:,10], axis=1)  # shape: (num_faces, 1) # 1 27
    hsv_hist_11 = np.expand_dims(np.asarray(hsv_hist)[:,11], axis=1)  # shape: (num_faces, 1) # 1 28
    hsv_hist_12 = np.expand_dims(np.asarray(hsv_hist)[:,12], axis=1)  # shape: (num_faces, 1) # 1 29
    hsv_hist_13 = np.expand_dims(np.asarray(hsv_hist)[:,13], axis=1)  # shape: (num_faces, 1) # 1 30
    hsv_hist_14 = np.expand_dims(np.asarray(hsv_hist)[:,14], axis=1)  # shape: (num_faces, 1) # 1 31
    hsv_hist_15 = np.expand_dims(np.asarray(hsv_hist)[:,15], axis=1)  # shape: (num_faces, 1) # 1 32
    hsv_hist_16 = np.expand_dims(np.asarray(hsv_hist)[:,16], axis=1)  # shape: (num_faces, 1) # 1 33
    hsv_hist_17 = np.expand_dims(np.asarray(hsv_hist)[:,17], axis=1)  # shape: (num_faces, 1) # 1 34
    hsv_hist_18 = np.expand_dims(np.asarray(hsv_hist)[:,18], axis=1)  # shape: (num_faces, 1) # 1 35
    hsv_hist_19 = np.expand_dims(np.asarray(hsv_hist)[:,19], axis=1)  # shape: (num_faces, 1) # 1 36
    hsv_hist_20 = np.expand_dims(np.asarray(hsv_hist)[:,20], axis=1)  # shape: (num_faces, 1) # 1 37
    hsv_hist_21 = np.expand_dims(np.asarray(hsv_hist)[:,21], axis=1)  # shape: (num_faces, 1) # 1 38
    hsv_hist_22 = np.expand_dims(np.asarray(hsv_hist)[:,22], axis=1)  # shape: (num_faces, 1) # 1 39
    hsv_hist_23 = np.expand_dims(np.asarray(hsv_hist)[:,23], axis=1)  # shape: (num_faces, 1) # 1 40
    hsv_hist_24 = np.expand_dims(np.asarray(hsv_hist)[:,24], axis=1)  # shape: (num_faces, 1) # 1 41


    # Triangle HSV stats
    triangle_hsv_stats_0 = np.expand_dims(np.asarray(triangle_hsv_stats)[:,0], axis=1)  # shape: (num_faces, 1) # 1 42
    triangle_hsv_stats_1 = np.expand_dims(np.asarray(triangle_hsv_stats)[:,1], axis=1)  # shape: (num_faces, 1) # 1 43 
    triangle_hsv_stats_2 = np.expand_dims(np.asarray(triangle_hsv_stats)[:,2], axis=1)  # shape: (num_faces, 1) # 1 44
    triangle_hsv_stats_3 = np.expand_dims(np.asarray(triangle_hsv_stats)[:,3], axis=1)  # shape: (num_faces, 1) # 1 45
    triangle_hsv_stats_4 = np.expand_dims(np.asarray(triangle_hsv_stats)[:,4], axis=1)  # shape: (num_faces, 1) # 1 46
    triangle_hsv_stats_5 = np.expand_dims(np.asarray(triangle_hsv_stats)[:,5], axis=1)  # shape: (num_faces, 1) # 1 47
    # 返回拼接后的特征
    mesh_features = np.concatenate((
        points, normal, area, area_mean, area_variance, flatness, density, verticality,
        masball_radius, masball_radius_mean, masball_radius_variance, height, mesh_hvi,
        hsv_hist_0, hsv_hist_1, hsv_hist_2, hsv_hist_3, hsv_hist_4, hsv_hist_5, hsv_hist_6,
        hsv_hist_7, hsv_hist_8, hsv_hist_9, hsv_hist_10, hsv_hist_11, hsv_hist_12, hsv_hist_13,
        hsv_hist_14, hsv_hist_15, hsv_hist_16, hsv_hist_17, hsv_hist_18, hsv_hist_19, hsv_hist_20,
        hsv_hist_21, hsv_hist_22, hsv_hist_23, hsv_hist_24, triangle_hsv_stats_0, triangle_hsv_stats_1,
        triangle_hsv_stats_2, triangle_hsv_stats_3, triangle_hsv_stats_4, triangle_hsv_stats_5
    ), axis=1)
    return mesh_features

skipped_files = []

for root, dirs, files in os.walk(obj_file_path):
    obj_files = [f for f in files if f.endswith(".obj")]
    for obj_file in tqdm(obj_files, desc="Processing OBJ files"):
        identifier = os.path.splitext(os.path.basename(obj_file))[0].rsplit('_', 1)[0]
        outh5_identifier = os.path.splitext(os.path.basename(obj_file))[0]
        out_h5file_path = os.path.join(output_dir, outh5_identifier + ".h5")

        if os.path.exists(out_h5file_path):
            print(f"Output H5 file for {identifier} already exists. Skipping.")
            continue

        try:
            mesh_fea = process_obj_feature(os.path.join(obj_file_path, outh5_identifier + '.obj'))
            if mesh_fea is None:
                raise ValueError("Feature extraction returned None")
        except Exception as e:
            print(f"Error processing {identifier}: {e}. Skipping file.")
            skipped_files.append(identifier)
            continue

        try:
            with h5py.File(out_h5file_path, 'w') as hf:
                hf.create_dataset('data', data=mesh_fea)
            print(f"Saved H5 file: {out_h5file_path}")
        except Exception as e:
            print(f"Error writing H5 for {obj_file}: {e}")
            continue

with open("skipped_files_train.txt", "w") as f:
    for fname in skipped_files:
        f.write(fname + "\n")


## 文件切分，避免文件过大模型无法读入数据
输入路径下需要留h5格式文件

In [ ]:
import os
import math
import h5py
import numpy as np
from collections import defaultdict

h5_root     = r"J:\paper_cfd_code\test"
output_base = r"J:\paper_cfd_code\test"
MIN_ROWS    = 81920     # 保留之前的最小行数阈值
MAX_BYTES   = 1 * 1024 * 1024  # 1 MB

def extract_scene_name(filename):
    base = os.path.splitext(filename)[0]
    tokens = base.split("_")
    if len(tokens) >= 3:
        return f"{tokens[1]}_{tokens[2]}_{tokens[-1]}"
    return base

def split_and_save_h5(h5_path, output_root):
    with h5py.File(h5_path, 'r') as f:
        data_array = f["data"][:]
    # 跳过过小的数据
    if data_array.shape[0] < MIN_ROWS:
        print(f"  [SKIP] {os.path.basename(h5_path)} has only {data_array.shape[0]} rows")
        return

    # 拆分份数由 color.npy 大小决定
    mesh_hvi       = data_array[:, 6:].astype(np.float32)
    total_bytes    = mesh_hvi.nbytes
    num_parts      = math.ceil(total_bytes / MAX_BYTES) if total_bytes > MAX_BYTES else 1

    scene_name = extract_scene_name(os.path.basename(h5_path))
    parts_data = np.array_split(data_array, num_parts)
    print(f"  Splitting into {num_parts} parts (color bytes={total_bytes})")

    for idx, data_part in enumerate(parts_data, start=1):
        subfolder_name = f"{scene_name}_{idx:03d}"
        subfolder_path = os.path.join(output_root, subfolder_name)
        os.makedirs(subfolder_path, exist_ok=True)

        points   = data_part[:, 0:3].astype(np.float32)
        normals  = data_part[:, 3:6].astype(np.float32)
        mesh_hvi = data_part[:, 6:].astype(np.float32)

        np.save(os.path.join(subfolder_path, "coord.npy"), points)
        np.save(os.path.join(subfolder_path, "normal.npy"), normals)
        np.save(os.path.join(subfolder_path, "color.npy"), mesh_hvi)

def rename_folders(root_dir):
    all_folders = sorted(os.listdir(root_dir))
    groups = defaultdict(list)
    for folder in all_folders:
        full_path = os.path.join(root_dir, folder)
        if os.path.isdir(full_path):
            parts = folder.split('_')
            group_key = '_'.join(parts[:-1]) if len(parts) >= 2 else folder
            groups[group_key].append(folder)

    new_names = {}
    for group_id, key in enumerate(sorted(groups.keys())):
        group_folders = groups[key]
        try:
            group_folders.sort(key=lambda x: int(x.split('_')[-1]))
        except ValueError:
            group_folders.sort()
        for intra_index, folder in enumerate(group_folders):
            new_names[folder] = f"scene{group_id:04d}_{intra_index:02d}"

    for old_name, new_name in new_names.items():
        old_path = os.path.join(root_dir, old_name)
        new_path = os.path.join(root_dir, new_name)
        print(f"Renaming: {old_name} -> {new_name}")
        os.rename(old_path, new_path)

if __name__ == "__main__":
    os.makedirs(output_base, exist_ok=True)
    print("=== Processing H5 files with dynamic split for color.npy <= 1MB ===")
    for file in sorted(os.listdir(h5_root)):
        if not file.lower().endswith(".h5"):
            continue
        print(f"Processing {file}")
        split_and_save_h5(os.path.join(h5_root, file), output_base)

    print("\nSplitting complete. Start renaming…")
    rename_folders(output_base)
    print("All done!")


## 植被颜色抖动
对植被颜色进行随机抖动，以增加数据的多样性和鲁棒性。<br>SUM数据集中植被颜色全部为绿色，这一现象会导致模型泛化性不足。<br>在SUM数据集中，植被的标签为2


In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import open3d as o3d
import matplotlib.pyplot as plt
from tqdm import tqdm
import random
import torchvision.transforms.functional as F
from sklearn.neighbors import NearestNeighbors
import trimesh
from PIL import Image, ImageOps
# =============================================================================
# 1. 根据OBJ文件名寻找对应的TXT文件，并加载点云及分类数据
# =============================================================================

def label_jitter(obj_file, txt_file_path, output_path, jitter_label):
    identifier = os.path.splitext(os.path.basename(obj_file))[0]
    output_objpath = os.path.join(output_path, identifier + "_jittered.obj")
    output_texturepath = os.path.join(identifier + "_jittered")  # 如有纹理，可用 JPG 或 PNG
    if os.path.exists(output_objpath):
            print(f"导出文件 {output_objpath} 已存在，跳过。")
            return
    def extract_patches_with_bbox(combined_img):
        # 构造二值掩膜（非黑区域）
        mask = np.any(combined_img != 0, axis=2).astype(np.uint8) * 255
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        patch_infos = []
        for cnt in contours:
            x, y, w, h = cv2.boundingRect(cnt)
            patch = combined_img[y:y+h, x:x+w]
            patch_infos.append((patch, x, y, w, h))
        return patch_infos

    class AsymmetricHueJitter:
        def __init__(self, hue_range):
            self.hue_min, self.hue_max = hue_range
        def __call__(self, img: Image.Image) -> Image.Image:
            hue_factor = random.uniform(self.hue_min, self.hue_max)
            return F.adjust_hue(img, hue_factor)
            
    matching_txt_files = []
    for root, dirs, files in os.walk(txt_file_path):
        for file in files:
            if file.endswith('.txt') and identifier in file:
                matching_txt_files.append(os.path.join(root, file))
    new_mesh = None

    # =============================================================================
    # 2. 针对每个匹配的TXT文件，提取class==2的三角形
    # =============================================================================
    if not matching_txt_files:
        print("没有匹配的 TXT 文件，直接返回。")
        return

    for txt_file in matching_txt_files:
        print(f"标签匹配中")
        with open(txt_file, "r") as f:
            lines = [line.strip() for line in f.readlines() if not line.startswith("//")]
        data = np.loadtxt(lines)
        df = pd.DataFrame(data, columns=['X_COG', 'Y_COG', 'Z_COG', 'classification'])
        point_cloud = df[['X_COG', 'Y_COG', 'Z_COG']].values
        labels = df['classification'].values

        # 读取OBJ网格
        read_mesh = o3d.io.read_triangle_mesh(obj_file)
        vertices = np.asarray(read_mesh.vertices)
        triangles = np.asarray(read_mesh.triangles)

        # 计算每个三角形的质心
        mesh_fea_center = np.mean(vertices[triangles], axis=1)

        # 利用最近邻为每个三角形质心分配标签
        nbrs = NearestNeighbors(n_neighbors=1, algorithm='auto').fit(point_cloud)
        distances, indices = nbrs.kneighbors(mesh_fea_center)
        triangle_labels = labels[indices.flatten()]

        # 提取标签为2的三角形索引
        veg_triangle = [i for i, label in enumerate(triangle_labels) if label == jitter_label]
        # print(f"共找到 {len(veg_triangle)} 个标签为 2 的三角形。")
        if len(veg_triangle) == 0:
            print("无标签为2的三角形，跳过可视化。")
            return

        # 从原网格中提取这些三角形及对应顶点
        veg_triangles = triangles[veg_triangle]
        veg_vertices_idx = np.unique(veg_triangles.flatten())
        veg_vertices = vertices[veg_vertices_idx]
        map_old2new = {old: new for new, old in enumerate(veg_vertices_idx)}
        veg_triangles_new = np.array([[map_old2new[idx] for idx in tri] for tri in veg_triangles], dtype=np.int32)

        # 构造新的网格对象（只包含class==2的三角形）
        mesh_veg = o3d.geometry.TriangleMesh()
        mesh_veg.vertices = o3d.utility.Vector3dVector(veg_vertices)
        mesh_veg.triangles = o3d.utility.Vector3iVector(veg_triangles_new)

        # 如果原网格有UV，则提取对应的纹理坐标
        if read_mesh.triangle_uvs is not None and len(read_mesh.triangle_uvs) > 0:
            orig_uvs = np.asarray(read_mesh.triangle_uvs)
            veg_uvs = []
            for t in veg_triangle:
                base_idx = t * 3
                veg_uvs.extend([orig_uvs[base_idx + 0], orig_uvs[base_idx + 1], orig_uvs[base_idx + 2]])
            veg_uvs = np.array(veg_uvs, dtype=np.float64)
            mesh_veg.triangle_uvs = o3d.utility.Vector2dVector(veg_uvs)

        # 复制材质索引（如果有）
        if read_mesh.triangle_material_ids is not None and len(read_mesh.triangle_material_ids) > 0:
            orig_mat_ids = np.asarray(read_mesh.triangle_material_ids)
            veg_mat_ids = orig_mat_ids[veg_triangle]
            mesh_veg.triangle_material_ids = o3d.utility.IntVector(veg_mat_ids.tolist())

        # 保留原始纹理贴图（这里使用纹理1）
        if read_mesh.textures is not None and len(read_mesh.textures) > 0:
            meshVeg_textures = []
            meshVeg_textures.append(read_mesh.textures[1])
            meshVeg_textures.append(read_mesh.textures[1])
            for i in range(2, len(read_mesh.textures) - 1):
                meshVeg_textures.append(read_mesh.textures[i])
            mesh_veg.textures = meshVeg_textures

        # =============================================================================
        # 3. 针对每个三角形提取纹理区域，并生成单独图片（仅保留三角形内纹理，其余为黑色）
        # =============================================================================
        # 构造纹理列表（假设read_mesh.textures中存放的是PIL.Image）
        mesh_textures = []
        mesh_textures.append(read_mesh.textures[1])
        mesh_textures.append(read_mesh.textures[1])
        for i in range(2, len(read_mesh.textures) - 1):
            mesh_textures.append(read_mesh.textures[i])

        # 先将每张纹理转换为HSV格式（便于后续处理）
        hsv_textures = []
        for texture in mesh_textures:
            texture_np = np.asarray(texture)
            texture_hsv = cv2.cvtColor(texture_np, cv2.COLOR_RGB2HSV)
            hsv_textures.append(texture_hsv)

        triangle_texture_images = []
        uvs_all = np.asarray(mesh_veg.triangle_uvs).reshape(-1, 3, 2)
        mat_ids_all = np.asarray(mesh_veg.triangle_material_ids)
        print('纹理拼接中')
        for i, (uv_coords, material_id) in enumerate(zip(uvs_all, mat_ids_all)):
            hsv_texture = hsv_textures[material_id]
            tex_h, tex_w = hsv_texture.shape[:2]
            pixel_coords = (uv_coords * [tex_w, tex_h]).astype(int)
            pixel_coords = np.clip(pixel_coords, 0, [tex_w - 1, tex_h - 1])
            min_x = np.min(pixel_coords[:, 0])
            max_x = np.max(pixel_coords[:, 0])
            min_y = np.min(pixel_coords[:, 1])
            max_y = np.max(pixel_coords[:, 1])
            cropped_hsv = hsv_texture[min_y:max_y+1, min_x:max_x+1]
            # 构造局部掩膜：仅保留三角形内的区域
            local_mask = np.zeros((max_y - min_y + 1, max_x - min_x + 1), dtype=np.uint8)
            local_pixel_coords = pixel_coords.copy()
            local_pixel_coords[:, 0] -= min_x
            local_pixel_coords[:, 1] -= min_y
            cv2.fillPoly(local_mask, [local_pixel_coords], 255)
            cropped_rgb = cv2.cvtColor(cropped_hsv, cv2.COLOR_HSV2RGB)
            black_background = np.zeros_like(cropped_rgb)
            black_background[local_mask == 255] = cropped_rgb[local_mask == 255]
            img_pil = Image.fromarray(black_background)
            triangle_texture_images.append(img_pil)

        # =============================================================================
        # 4. 将每个三角形纹理粘贴回对应材质的全黑纹理图中
        # =============================================================================
        material_ids = np.asarray(mesh_veg.triangle_material_ids)
        unique_mat_ids = np.unique(material_ids)
        combined_textures = {}
        for mat_id in unique_mat_ids:
            orig_texture_np = np.asarray(mesh_textures[mat_id])
            h, w = orig_texture_np.shape[:2]
            combined_textures[mat_id] = np.zeros((h, w, 3), dtype=np.uint8)

        uvs_all = np.asarray(mesh_veg.triangle_uvs).reshape(-1, 3, 2)
        mat_ids_all = np.asarray(mesh_veg.triangle_material_ids)
        for i, (uv_coords, mat_id) in enumerate(zip(uvs_all, mat_ids_all)):
            orig_texture_np = np.asarray(mesh_textures[mat_id])
            tex_h, tex_w = orig_texture_np.shape[:2]
            pixel_coords = (uv_coords * [tex_w, tex_h]).astype(int)
            pixel_coords = np.clip(pixel_coords, 0, [tex_w - 1, tex_h - 1])
            min_x = np.min(pixel_coords[:, 0])
            max_x = np.max(pixel_coords[:, 0])
            min_y = np.min(pixel_coords[:, 1])
            max_y = np.max(pixel_coords[:, 1])
            tri_img = np.array(triangle_texture_images[i])
            # 生成非黑区域掩膜
            mask = (tri_img.sum(axis=2) > 0)
            target_region = combined_textures[mat_id][min_y:max_y+1, min_x:max_x+1]
            if target_region.shape[:2] != tri_img.shape[:2]:
                print(f"Warning: Region shape {target_region.shape[:2]} differs from triangle image shape {tri_img.shape[:2]} at triangle {i}.")
                continue
            target_region[mask] = tri_img[mask]
            combined_textures[mat_id][min_y:max_y+1, min_x:max_x+1] = target_region

        # =============================================================================
        # 5. 对拼接后的纹理图提取图斑，并对每个图斑进行色调扰动
        # =============================================================================
        print('纹理扰动中')

        # 这里以第一个材质的拼接图为例
        combined_img = np.array(list(combined_textures.values())[0])
        patch_infos = extract_patches_with_bbox(combined_img)
        

        asymmetric_jitter = AsymmetricHueJitter(hue_range=(-0.3, 0.15))
        jittered_patch_infos = []
        for (patch, x, y, w, h) in patch_infos:
            patch_pil = Image.fromarray(patch)
            jittered_img = asymmetric_jitter(patch_pil)
            jittered_patch = np.array(jittered_img)
            jittered_patch_infos.append((jittered_patch, x, y, w, h))

        # =============================================================================
        # 6. 重新拼接扰动后的图斑到大图中，注意只替换非黑区域
        # =============================================================================
        reassembled_img = np.asarray(read_mesh.textures[1])
        for (jittered_patch, x, y, w, h) in jittered_patch_infos:
            # 生成非黑区域掩膜
            mask = (jittered_patch.sum(axis=2) > 0)
            region = reassembled_img[y:y+h, x:x+w]
            region[mask] = jittered_patch[mask]
            reassembled_img[y:y+h, x:x+w] = region

        new_mesh = o3d.geometry.TriangleMesh()
        new_mesh.vertices = read_mesh.vertices
        new_mesh.triangles = read_mesh.triangles
        new_mesh.triangle_material_ids = read_mesh.triangle_material_ids
        new_mesh.triangle_uvs = read_mesh.triangle_uvs
        # =============================================================================
        # 7. 将扰动后的纹理更新到网格中并可视化
        # =============================================================================
        # 确保图像为uint8类型
        if reassembled_img.dtype != np.uint8:
            reassembled_img = (reassembled_img * 255).astype(np.uint8)
        texture_o3d = o3d.geometry.Image(reassembled_img)
        new_mesh.textures = [texture_o3d, texture_o3d]
        new_mesh.compute_vertex_normals()
        # print("使用扰动后的纹理可视化网格模型...")
    print('数据导出中')
    export_vertices = np.asarray(new_mesh.vertices)
    export_faces = np.asarray(new_mesh.triangles)
    export_mesh = trimesh.Trimesh(vertices=export_vertices, faces=export_faces, process=False)

    uvs_per_triangle = np.asarray(new_mesh.triangle_uvs)  # 每个三角形的 3 个 UV
    uvs_per_vertex = np.zeros((export_vertices.shape[0], 2))  # 创建 per-vertex UV 容器

    # 遍历 faces，把每个 face 的 UV 映射到对应的 vertex
    tri_vertex_indices = export_faces.flatten()  # 获取所有顶点索引 (3 * n_faces)
    uvs_per_vertex[tri_vertex_indices] = uvs_per_triangle  # 直接映射 UV
    export_uvs = uvs_per_vertex
    texture_np = np.asarray(new_mesh.textures[0])  # Open3D 纹理转换为 numpy
    im = Image.fromarray(texture_np)
    export_img = ImageOps.flip(im)
    texture_visuals = trimesh.visual.texture.TextureVisuals(uv=export_uvs, image=export_img)
    export_mesh.visual = texture_visuals  # 绑定到 trimesh 对象
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    
    
    # 如果网格有材质信息，修改材质名称和纹理文件名
    if hasattr(export_mesh.visual, 'material') and export_mesh.visual.material is not None:
        export_mesh.visual.material.name = output_texturepath
        # export_mesh.visual.material.image_name = os.path.basename(output_texturepath)

    # 导出 OBJ 文件（trimesh 会自动生成对应的 MTL 文件，并写入材质信息）
    try:
        print("正在导出文件...")
        export_mesh.export(output_objpath, file_type='obj', mtl_name=identifier + "_jittered.mtl")
    except Exception as e:
        print(f"导出文件出错: {output_objpath}\n错误信息: {e}")
    if new_mesh is None:
        print("未生成有效的 new_mesh，跳过导出。")
        return
    
if __name__ == "__main__":
    obj_filepath = r'D:\data_cfd\SUM_Helsinki_C6_mesh\SUM_Helsinki_C6_mesh\test_obj'
    txt_file_path = r'D:\data_cfd\SUM_Helsinki_C6_mesh\SUM_Helsinki_C6_mesh\test_obj'
    output_path = r'D:\data_cfd\SUM_Helsinki_C6_mesh\SUM_Helsinki_C6_mesh\test_jitter'
    jitter_label = 2

    abs_output = os.path.abspath(output_path)
# o3d.io.
    # 第一步：遍历目录树，收集所有待处理的 OBJ 文件
    obj_files = []
    for root, dirs, files in os.walk(obj_filepath):
        # 如果当前目录在输出目录中，则跳过
        if os.path.abspath(root).startswith(abs_output):
            print("输出与搜索路径发生嵌套，将不对生成的文件进行处理。")
            continue
        for file in files:
            if file.endswith('.obj') and "_jittered" not in file:
                obj_files.append(os.path.join(root, file))

    # 第二步：使用全局进度条处理所有收集到的 OBJ 文件
    for obj_file in tqdm(obj_files, desc="Processing each OBJ file", total=len(obj_files)):
        print('正在处理', obj_file)
        label_jitter(obj_file, txt_file_path, output_path, jitter_label)